<a href="https://colab.research.google.com/github/nevilshah235/HackerEarth_Gala_Image_Challenge/blob/master/hackerearth_gala_image_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Load Data 

1. Define a function to load the Gala Image Data

In [0]:
import pdb
from os.path import join
from tensorflow.keras.preprocessing.image import load_img, img_to_array

train = pd.read_csv('/kaggle/input/galasimages/dataset/train.csv')
test = pd.read_csv('/kaggle/input/galasimages/dataset/test.csv')

train_img_path = '/kaggle/input/galasimages/dataset/Train Images/'
test_img_path = '/kaggle/input/galasimages/dataset/Test Images/'

labels = ['food', 'attire', 'decor and signage', 'miscellaneous']
img_size = 224

Labels = {'Food':0, 'Attire':1,'Decorationandsignage':2 ,'misc':3}

def load_images(img_path, img_height=img_size, img_width=img_size):
        ImgList = os.listdir(img_path)
        Images = [load_img(join(img_path + img), target_size = (img_height, img_width)) for img in ImgList]
        img_array = np.array([img_to_array(img) for img in Images])/255.0
        if(img_path == train_img_path):
            y = [Labels[list(train[img == train.Image].Class)[0]] for img in ImgList]
        else:
            y = ImgList
        return img_array,y

Train_images, y_train = load_images(train_img_path, img_size, img_size)



In [0]:
from sklearn.preprocessing import OneHotEncoder
OH_encoder = OneHotEncoder()
pd_y_train = pd.DataFrame(y_train, columns = ['Label'])
OH_y_train = OH_encoder.fit_transform(pd_y_train).toarray()
#train.head()
#OH_y_train
#list(train['image7042.jpg' == train.Image].Class)[0]
#print(os.listdir(train_img_path))

In [0]:
from tensorflow import keras
from keras.models import Sequential
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

num_classes = 4

resnet50 = ResNet50(include_top=False, weights='imagenet',input_shape = (img_size, img_size, 3), pooling='avg')

resnet50.trainable=False

Model=Sequential()
Model.add(resnet50)

Model.add(Dense(1024,activation='relu'))
Model.add(Dense(512,activation='relu'))
Model.add(Dense(num_classes,activation='softmax'))
Model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

num_classes = 4

reduce_learning_rate = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, cooldown = 2, min_lr = 0.00001, verbose=1)
#es = EarlyStopping(monitor='val_accuracy', mode='max', baseline=0.9, verbose=1)
#cb = [es]
cb = []
xception = Xception(include_top=False, weights='imagenet',input_shape = (img_size, img_size, 3), pooling='avg')

xception.trainable=False

Model=Sequential()
Model.add(xception)

Model.add(Dense(1024,activation='relu'))
Model.add(Dropout(0.5))
#Model.add(Dense(256,activation='relu'))
Model.add(Dense(num_classes,activation='softmax'))
Model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

num_classes = 4

reduce_learning_rate = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, cooldown = 2, min_lr = 0.00001, verbose=1)
#es = EarlyStopping(monitor='val_accuracy', mode='max', baseline=0.9)
cb = [reduce_learning_rate]

vgg19 = VGG19(include_top=False, weights='imagenet',input_shape = (img_size, img_size, 3), pooling='avg')

vgg19.trainable=False

Model=Sequential()
Model.add(vgg19)

Model.add(Dense(1024,activation='relu'))
Model.add(Dropout(0.5))
Model.add(Dense(num_classes,activation='softmax'))
Model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D,Dropout

num_classes = 4
Model = Sequential()
Model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu', input_shape = (img_size, img_size, 3)))
Model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
Model.add(MaxPooling2D(pool_size=(2, 2),strides = (2,2)))
Model.add(Conv2D(128, kernel_size = (3,3), activation = 'relu'))
Model.add(Conv2D(128, kernel_size = (3,3), activation = 'relu'))
Model.add(MaxPooling2D(pool_size=(2, 2),strides = (2,2)))
Model.add(Conv2D(256, kernel_size = (3,3), activation = 'relu'))
Model.add(Conv2D(256, kernel_size = (3,3), activation = 'relu'))
Model.add(MaxPooling2D(pool_size=(2, 2),strides = (2,2)))
Model.add(Conv2D(512, kernel_size = (3,3), activation = 'relu'))
Model.add(Conv2D(512, kernel_size = (3,3), activation = 'relu'))
Model.add(MaxPooling2D(pool_size=(2, 2),strides = (2,2)))
Model.add(Flatten())
Model.add(Dense(1024, activation = 'relu'))
Model.add(Dense(num_classes, activation = 'softmax'))
Model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
#Model.fit(Train_images, OH_y_train, batch_size = 200, epochs = 3, validation_split = 0.2)


In [0]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32
X_train, X_val, Y_train, Y_val = train_test_split(Train_images,OH_y_train, test_size=0.2, random_state=1)
train_datagen = ImageDataGenerator(rotation_range=10,zoom_range = 0.20, width_shift_range=0.20,height_shift_range=0.20,horizontal_flip=True)
train_generator = train_datagen.flow(np.array(X_train), Y_train, batch_size=batch_size)
#val_datagen = ImageDataGenerator(rotation_range=10,zoom_range = 0.20, width_shift_range=0.20,height_shift_range=0.20,horizontal_flip=True)
#val_generator = train_datagen.flow(np.array(X_val), Y_val, batch_size=batch_size)
history = Model.fit_generator(train_generator, validation_data=(X_val, Y_val), epochs = 100, callbacks = cb)


In [0]:
Train_images = 0

In [0]:
Test_images, test_name = load_images(test_img_path, img_size, img_size)
Test_labels = Model.predict(Test_images)

In [0]:
#print(Test_labels)
inv_Labels = {0:'Food', 1:'Attire',2:'Decorationandsignage' ,3:'misc'}
Test_Labels = [inv_Labels[np.argmax(il)] for il in Test_labels]
submission = pd.DataFrame({ 'Image': test_name, 'Class': Test_Labels })
submision = submission.sort_values('Image')
submission.to_csv('submission.csv', index=False)

In [0]:
len(Test_labels)